In [1]:
import folium
import armageddon

# Initial value, You should use results from previous modules as the value of these parameters
burst_lat = 51.72
burst_lon = -3.8
blast_lat = 51.716033
blast_lon = -3.845419
radius_list = [0.3e3, 0.5e3]

# The PostcodeLocator tool
locator = armageddon.PostcodeLocator()

# Unit
postcodes = locator.get_postcodes_by_radius((blast_lat, blast_lon), radius_list)   
population = locator.get_population_of_postcode(postcodes)
armageddon.plot_results(burst_lat, burst_lon, blast_lat, blast_lon, radius_list, postcodes, population)


In [2]:
# Alternatively find the postcode sectors in the damage radii, and populations of the sectors
sectors = locator.get_postcodes_by_radius((blast_lat, blast_lon), radius_list, sector=True)
# print(sectors)

# for sector, the results from get_postcodes should be added one space for each postcodes before passing to get_population
for i in range(len(sectors)):
    for j in range(len(sectors[i])):
        sectors[i][j] = sectors[i][j][:4] + ' ' + sectors[i][j][4:]

population_sector = locator.get_population_of_postcode(sectors, sector=True)
# print(population_sector)
armageddon.plot_results(burst_lat, burst_lon, blast_lat, blast_lon, radius_list, sectors, population_sector, sector = True)